In [2]:
from bs4 import BeautifulSoup
import urllib2
import string
import sqlite3
from functools import reduce

In [3]:
def scrape_page(url, c, ID):
    try:
        page = urllib2.urlopen("https://www.baseball-reference.com" + url)
        soup = BeautifulSoup(page, 'lxml')
    except:
        pass # The HTML was improper
    try:
        batting = soup.findAll("div", {"id" : "all_batting_standard"})[0]
        batting_years = batting.findAll("tr", id=lambda x: x and x.startswith('batting_standard.'))
        # https://stackoverflow.com/questions/2830530/matching-ids-in-beautifulsoup

        # Handles all single years of batting statistics
        count = 1
        league = ["0", "0", "0", "0", "0", "0"]
        single_league = ""
        teams = set()
        for year in batting_years:
            # Format the year of statistics and add it to the database
            values = str(count) + ", "
            values += year.find("th").text + ", "
            stats = [i.text for i in year.findAll("td")]
            values += stats[0] + ", " if stats[0] != "--" else "'NONE', "
            values += "'" + stats[1] + "', "
            values += "'" + stats[2] + "', "
            values += ", ".join([stats[i] if len(stats[i]) != 0 else "'NONE'" for i in range(3, len(stats) - 2)])
            values += ", '" + stats[-2] + "', "
            values += "'" + str(stats[-1]) + "'" if len(stats[-1]) != 0 else "'NONE'"
            c.execute("INSERT INTO season_batting_stats VALUES (" + str(ID) + ", " + values + ")")
            
            # Add to the teams set to keep track if only one team was played for or if multiple were for career stats
            teams.add("'" + stats[1] + "'")
            
            # Add which league was played in to be able to put it into the database
            if stats[2] == "AL":
                league[0] = "1"
                single_league = "'AL'"
            elif stats[2] == "NL":
                league[1] = "1"
                single_league = "'NL'"
            elif stats[2] == "AA":
                league[2] = "1"
                single_league = "'AA'"
            elif stats[2] == "NA":
                league[3] = "1"
                single_league = "'NA'"
            elif stats[2] == "UA":
                league[4] = "1"
                single_league = "'UA'"
            elif stats[2] == "PL":
                league[5] = "1"
                single_league = "'PL'"
            
            count += 1
            
        # Add data to the league database   
        c.execute("INSERT INTO league VALUES (" + str(ID) + ", " + ", ".join(league) + ")") 
        
        # Adds data for what the player did in his whole career
        totals = batting.findAll("tfoot")[0].findAll("tr")
        idx = 0
        add = True
        for total in totals:
            # The try discerns whether a row is a spacer or actually contains statistics
            try:
                total['class']
                idx += 1
            except:
                if add and idx == 0:
                    values = total.find("th").text.split(" ")[0] + ", " + ", ".join([i.text if len(i.text) != 0 else "'NONE'" for i in total.findAll("td")][:-2]) 
                    # -2 removes position and awards
                    c.execute("INSERT INTO career_batting_stats VALUES (" + str(ID) + ", " + values + ")")
                    
                    # Only played for 1 league so the stats wont be divided on baseball-reference
                    if sum(map(int, league)) == 1: 
                        c.execute("INSERT INTO league_batting_stats VALUES (" + str(ID) + ", " + single_league + ", "+ values + ")")
                    
                    # Only played for 1 team so the stats wont be divided on baseball-reference
                    if len(teams) == 1:
                        team = teams.pop()
                        c.execute("INSERT INTO team_batting_stats VALUES (" + str(ID) + ", " + team + ", "+ values + ")")
                        c.execute("INSERT INTO teams VALUES (" + str(ID) + ", " + team + ")") 
                    add = False
                
                # The player played for multiple teams and maybe multiple leagues, so that has to be handled properly
                elif idx != 0:
                    header = total.find("th").text.split(" (")
                    header[0] = "'" + header[0] + "'"
                    header[1] = header[1].split(" ")[0]
                    values =  ", ".join(header)+ ", " + ", ".join([i.text if len(i.text) != 0 else "'NONE'" for i in total.findAll("td")][:-2]) 
                    #Played for multiple teams and maybe multiple leagues
                    if idx == 1:
                        c.execute("INSERT INTO team_batting_stats VALUES (" + str(ID) + ", " + values + ")")
                        c.execute("INSERT INTO teams VALUES (" + str(ID) + ", " + header[0] + ")") 
                    else:
                        c.execute("INSERT INTO league_batting_stats VALUES (" + str(ID) + ", " + values + ")")           
    except:
        pass

In [4]:
def populate_table():
    #https://docs.python.org/2/library/sqlite3.html Used for sqlite3 syntax below
    conn = sqlite3.connect('/Users/jberman/Desktop/player_data.db')
    c = conn.cursor()
    count = 1
    for letter in string.lowercase: #https://stackoverflow.com/questions/16060899/alphabet-range-python

        print letter + " starting " + str(count)
        # Open the page of players for each letter
        page = urllib2.urlopen("https://www.baseball-reference.com/players/" + letter + "/")
        soup = BeautifulSoup(page, 'lxml') 

        # Get a list of all the players for the letter, scrape their statistics, and add it to the database
        players = soup.findAll("div", { "id" : "div_players_" })[0]
        players_formatted = players.findAll('a', href=True)

        for player in players_formatted:
            name = player.text.replace("'","") #https://stackoverflow.com/questions/29012820/python-remove-all-apostrophes-from-a-string
            c.execute("INSERT INTO players(Name) VALUES ('" + name + "')")
            scrape_page(player['href'], c, count)
            
            # Save every 100 players in case of error while running
            if count % 100 == 1:
                conn.commit()
                conn.close()
                print letter + " committed " + str(count)
                conn = sqlite3.connect('/Users/jberman/Desktop/player_data.db')
                c = conn.cursor()

            count += 1
        
        

    # Save (commit) the changes
    conn.commit()
    conn.close()
    print letter + " committed"

In [5]:
def make_table():
    conn = sqlite3.connect('/Users/jberman/Desktop/player_data.db')
    c = conn.cursor()
    
    # How to implement foreign keys https://sqlite.org/foreignkeys.html
    
    # Create player ID table
    c.execute("CREATE TABLE players (ID Integer PRIMARY KEY, Name varchar(255) NOT NULL)")

    # Create teams played on table
    c.execute("CREATE TABLE teams (" +
              "ID Integer NOT NULL," +
              "Team varchar(255) NOT NULL," + 
              "FOREIGN KEY(ID) REFERENCES players(ID))")

    # Create leagues played in table. 1 is true 0 is false
    c.execute("CREATE TABLE league (" +
              "ID Integer NOT NULL," +
              "AL Integer NOT NULL," + 
              "NL Integer NOT NULL," + 
              "AA Integer NOT NULL," + 
              "NA Integer NOT NULL," + 
              "UA Integer NOT NULL," + 
              "PL Integer NOT NULL," + 
              "FOREIGN KEY(ID) REFERENCES players(ID))")

    # Make the table for batting stats in a season
    c.execute("CREATE TABLE season_batting_stats (" + 
              "ID Integer NOT NULL," + 
              "Season Integer NOT NULL," + 
              "Year Integer NOT NULL," + 
              "Age Integer NOT NULL," + 
              "Team Text NOT NULL," + 
              "League Text NOT NULL," + 
              "Games Integer NOT NULL," + 
              "Plate_Appearances Integer NOT NULL," + 
              "At_Bats Integer NOT NULL," + 
              "Runs Integer NOT NULL," + 
              "Hits Integer NOT NULL," + 
              "Doubles Integer NOT NULL," + 
              "Triples Integer NOT NULL," + 
              "Home_Runs Integer NOT NULL," + 
              "RBIs Integer NOT NULL," + 
              "Stolen_Bases Integer NOT NULL," + 
              "Caught_Stealing Integer NOT NULL," + 
              "Walks Integer NOT NULL," + 
              "Strike_Outs Integer NOT NULL," + 
              "Batting_Average Real NOT NULL," + 
              "On_Base_Percentage Real NOT NULL," + 
              "Slugging Real NOT NULL," + 
              "OPS Real NOT NULL," + 
              "OPS_Plus Real NOT NULL," + 
              "Total_Bases Integer NOT NULL," + 
              "Double_Plays_Grounded_Into Integer NOT NULL," + 
              "Hit_By_Pitch Integer NOT NULL," + 
              "Sacrifice_Hits Integer NOT NULL," + 
              "Sacrifice_Flies Integer NOT NULL," + 
              "Intentional_Bases_On_Balls Integer NOT NULL," + 
              "Position varchar(255) NOT NULL," + 
              "Awards varchar(255)," +
              "FOREIGN KEY(ID) REFERENCES players(ID))")

    # Make the table for career batting stats
    c.execute("CREATE TABLE career_batting_stats (" + 
              "ID Integer NOT NULL," + 
              "Number_Of_Seasons Integer NOT NULL," + 
              "Games Integer NOT NULL," + 
              "Plate_Appearances Integer NOT NULL," + 
              "At_Bats Integer NOT NULL," + 
              "Runs Integer NOT NULL," + 
              "Hits Integer NOT NULL," + 
              "Doubles Integer NOT NULL," + 
              "Triples Integer NOT NULL," + 
              "Home_Runs Integer NOT NULL," + 
              "RBIs Integer NOT NULL," + 
              "Stolen_Bases Integer NOT NULL," + 
              "Caught_Stealing Integer NOT NULL," + 
              "Walks Integer NOT NULL," + 
              "Strike_Outs Integer NOT NULL," + 
              "Batting_Average Real NOT NULL," + 
              "On_Base_Percentage Real NOT NULL," + 
              "Slugging Real NOT NULL," + 
              "OPS Real NOT NULL," + 
              "OPS_Plus Real NOT NULL," + 
              "Total_Bases Integer NOT NULL," + 
              "Double_Plays_Grounded_Into Integer NOT NULL," + 
              "Hit_By_Pitch Integer NOT NULL," + 
              "Sacrifice_Hits Integer NOT NULL," + 
              "Sacrifice_Flies Integer NOT NULL," + 
              "Intentional_Bases_On_Balls Integer NOT NULL," + 
              "FOREIGN KEY(ID) REFERENCES players(ID))")

    # Make the table for career team batting stats
    c.execute("CREATE TABLE team_batting_stats (" + 
              "ID Integer NOT NULL," + 
              "Team Text NOT NULL," + 
              "Number_Of_Seasons Integer NOT NULL," + 
              "Games Integer NOT NULL," + 
              "Plate_Appearances Integer NOT NULL," + 
              "At_Bats Integer NOT NULL," + 
              "Runs Integer NOT NULL," + 
              "Hits Integer NOT NULL," + 
              "Doubles Integer NOT NULL," + 
              "Triples Integer NOT NULL," + 
              "Home_Runs Integer NOT NULL," + 
              "RBIs Integer NOT NULL," + 
              "Stolen_Bases Integer NOT NULL," + 
              "Caught_Stealing Integer NOT NULL," + 
              "Walks Integer NOT NULL," + 
              "Strike_Outs Integer NOT NULL," + 
              "Batting_Average Real NOT NULL," + 
              "On_Base_Percentage Real NOT NULL," + 
              "Slugging Real NOT NULL," + 
              "OPS Real NOT NULL," + 
              "OPS_Plus Real NOT NULL," + 
              "Total_Bases Integer NOT NULL," + 
              "Double_Plays_Grounded_Into Integer NOT NULL," + 
              "Hit_By_Pitch Integer NOT NULL," + 
              "Sacrifice_Hits Integer NOT NULL," + 
              "Sacrifice_Flies Integer NOT NULL," + 
              "Intentional_Bases_On_Balls Integer NOT NULL," + 
              "FOREIGN KEY(ID) REFERENCES players(ID))")

    # Make the table for career league batting stats
    c.execute("CREATE TABLE league_batting_stats (" + 
              "ID Integer NOT NULL," + 
              "League Text NOT NULL," + 
              "Number_Of_Seasons Integer NOT NULL," + 
              "Games Integer NOT NULL," + 
              "Plate_Appearances Integer NOT NULL," + 
              "At_Bats Integer NOT NULL," + 
              "Runs Integer NOT NULL," + 
              "Hits Integer NOT NULL," + 
              "Doubles Integer NOT NULL," + 
              "Triples Integer NOT NULL," + 
              "Home_Runs Integer NOT NULL," + 
              "RBIs Integer NOT NULL," + 
              "Stolen_Bases Integer NOT NULL," + 
              "Caught_Stealing Integer NOT NULL," + 
              "Walks Integer NOT NULL," + 
              "Strike_Outs Integer NOT NULL," + 
              "Batting_Average Real NOT NULL," + 
              "On_Base_Percentage Real NOT NULL," + 
              "Slugging Real NOT NULL," + 
              "OPS Real NOT NULL," + 
              "OPS_Plus Real NOT NULL," + 
              "Total_Bases Integer NOT NULL," + 
              "Double_Plays_Grounded_Into Integer NOT NULL," + 
              "Hit_By_Pitch Integer NOT NULL," + 
              "Sacrifice_Hits Integer NOT NULL," + 
              "Sacrifice_Flies Integer NOT NULL," + 
              "Intentional_Bases_On_Balls Integer NOT NULL," + 
              "FOREIGN KEY(ID) REFERENCES players(ID))")
    
    # Save (commit) the changes
    conn.commit()
    conn.close()

In [6]:
def drop_table():
    conn = sqlite3.connect('/Users/jberman/Desktop/player_data.db')
    c = conn.cursor()

    c.execute("DROP TABLE players")
    c.execute("DROP TABLE teams")
    c.execute("DROP TABLE league")
    c.execute("DROP TABLE season_batting_stats")
    c.execute("DROP TABLE career_batting_stats")
    c.execute("DROP TABLE team_batting_stats")
    c.execute("DROP TABLE league_batting_stats")

    # Save (commit) the changes
    conn.commit()
    conn.close()

In [7]:
def create_table():
    drop_table()
    make_table()

In [8]:
create_table()
populate_table()

w starting 18102
18102
18103
18104
18105
18106
18107
18108
18109
18110
18111
18112
18113
18114
18115
18116
18117
18118
18119
18120
18121
18122
18123
18124
18125
18126
18127
18128
18129
18130
18131
18132
18133
18134
18135
18136
18137
18138
18139
18140
18141
18142
18143
18144
18145
18146
18147
18148
18149
18150
18151
18152
18153
18154
18155
18156
18157
18158
18159
18160
18161
18162
18163
18164
18165
18166
18167
18168
18169
18170
18171
18172
18173
18174
18175
18176
18177
18178
18179
18180
18181
18182
18183
18184
18185
18186
18187
18188
18189
18190
18191
18192
18193
18194
18195
18196
18197
18198
18199
18200
18201
w committed 18201
18202
18203
18204
18205
18206
18207
18208
18209
18210
18211
18212
18213
18214
18215
18216
18217
18218
18219
18220
18221
18222
18223
18224
18225
18226
18227
18228
18229
18230
18231
18232
18233
18234
18235
18236
18237
18238
18239
18240
18241
18242
18243
18244
18245
18246
18247
18248
18249
18250
18251
18252
18253
18254
18255
18256
18257
18258
18259
18260
18261
18262